# DIMOS: Homopolymer Example

The following notebook demonstrates how a simple simulation of a homopolymer can be set up.
We exemplify this for a single chain in vacuum and a chain embedded in a Lennard-Jones fluid.

This is example code to showcase the modularity of DIMOS, and has not been thoroughly tested.

In [1]:
import dimos
import torch
import math

Specify the system and simulation parameters

In [2]:
num_monomers = 50
periodic = True
use_neighborlist = True
explicit_solvent = True
density = 0.5
simulation_temperature = 0.5

# These are the bonded parameters
r_0 = 0.7
K=2


We first set up the minimal system by heuristically determining the simulation box size and setting the masses to ones and charges to zero.

In [3]:
linear_size_box = 5.0*num_monomers**(3.0/5.0)

if explicit_solvent:
    num_solvent = math.ceil(linear_size_box**3*density*0.1)
else:
    num_solvent = 0

system = dimos.MinimalSystem("unitless")
system.num_atoms = num_monomers + num_solvent

system.masses = torch.ones(system.num_atoms)
system.charges = torch.zeros(system.num_atoms)
system.box = torch.tensor([linear_size_box,linear_size_box,linear_size_box])
system.periodic = periodic
system.use_neighborlist = use_neighborlist

In [4]:
bond_list = []
bond_parameters = []
for i in range(num_monomers-1):
    bond_list.append([i,i+1])
    bond_parameters.append([K,r_0])

positions = torch.zeros((system.num_atoms, 3))

#initialize by (pseudo) random walk
positions[0, :] = 0
for monomer in range(1, num_monomers):
    new_pos = positions[monomer-1, :] + torch.rand(1,3) * 2 * r_0 
    positions[monomer, :] = new_pos 

for solvent in range(num_solvent):
    positions[num_monomers + solvent] = torch.rand((1,3))*linear_size_box

We now set the bond parameters between bonded atoms and note them to be excluded from the non-bonded interactions.

In [5]:
bond_list = torch.tensor(bond_list).T
bond_parameters = torch.tensor(bond_parameters)

system.bonded_force_components.append(dimos.HarmonicBond(bond_list, bond_parameters, periodic, system.box))
system.all_exclusions = bond_list

Next, we define the nonbonded parameters; here we make a distinction of whether we need to restrict the interactions more specifically than over the system.all_exclusions above. This is the case if one, such as in our case with explicit solvent, wants to have different cutoffs for different contributions.

In [6]:
sigmas = torch.ones(system.num_atoms)*r_0/2**(1.0/6.0)
epsilons = torch.ones(system.num_atoms)

system.cutoff = 2.5*sigmas[0]

if explicit_solvent:
    atoms = torch.arange(sigmas.numel())
    apply_to_pairs = torch.cartesian_prod(atoms, atoms)

    nonbonded_energy = dimos.energy.NonbondedLennardJonesCG(system.num_atoms, sigmas, epsilons, system.cutoff, switch_distance=None, periodic=system.periodic, box=system.box, applies_to=apply_to_pairs)
else:
    nonbonded_energy = dimos.energy.NonbondedLennardJonesCG(system.num_atoms, sigmas, epsilons, system.cutoff, switch_distance=None, periodic=system.periodic, box=system.box)
system.nonbonded_force_components.append(nonbonded_energy)

if explicit_solvent:
    solvent_atoms = torch.arange(num_monomers, num_solvent)
    solvent_monomer_pairs = torch.cartesian_prod(atoms, solvent_atoms)
    monomer_monomer_pairs = torch.cartesian_prod(solvent_atoms, solvent_atoms)

    resulting_pairs = torch.cat((solvent_monomer_pairs, monomer_monomer_pairs))

    cutoff = 2**(1.0/6.0)*sigmas[0]
    solvent_nonbonded_energy = dimos.energy.NonbondedLennardJonesCG(system.num_atoms, sigmas, epsilons, cutoff, switch_distance=None, periodic=system.periodic, box=system.box, applies_to=resulting_pairs)
system.nonbonded_force_components.append(solvent_nonbonded_energy)

We are nearly done... Now we set up the integrator, and initialize the simulation object.

In [7]:
tau_0 = (system.masses[0]*sigmas[0]**2/epsilons[0])**0.5

integrator = dimos.AndersenDynamics(0.005*tau_0, simulation_temperature, 0.1, system)
#VelocityVerlet(0.005*tau_0, system)
simulation = dimos.MDSimulation(system, integrator, positions, temperature=simulation_temperature)

And run the simulation by writing out the positions and printing the energy contributions of the atoms to a file every 100 MD steps.

In [8]:
config_output = open("config.dat", "w")
for _ in range(100):
    for idx, pos in enumerate(simulation.pos):
        if idx <= num_monomers:
            print(f"{pos[0]}\t{pos[1]}\t{pos[2]}\t1", file=config_output)
        else:
            print(f"{pos[0]}\t{pos[1]}\t{pos[2]}\t0", file=config_output)
    print(file=config_output)
    print(file=config_output, flush=True) 
    if explicit_solvent:
        print(simulation.sys.bonded_force_components[0].calc_energy(simulation.pos), simulation.sys.nonbonded_force_components[0].calc_energy(simulation.pos, simulation.neighborlist), simulation.sys.nonbonded_force_components[1].calc_energy(simulation.pos, simulation.neighborlist))
    else:
        print(simulation.sys.bonded_force_components[0].calc_energy(simulation.pos), simulation.sys.nonbonded_force_components[0].calc_energy(simulation.pos, simulation.neighborlist))        
    simulation.step(100)
    

tensor(51.9600, grad_fn=<SumBackward0>) tensor(1.0543e+10, grad_fn=<AddBackward0>) tensor(1.0543e+10, grad_fn=<AddBackward0>)
tensor(2845.6267, grad_fn=<SumBackward0>) tensor(-550.0425, grad_fn=<AddBackward0>) tensor(-81.1047, grad_fn=<AddBackward0>)
tensor(2669.6650, grad_fn=<SumBackward0>) tensor(-653.5353, grad_fn=<AddBackward0>) tensor(-99.4335, grad_fn=<AddBackward0>)
tensor(6798.8613, grad_fn=<SumBackward0>) tensor(28460.8652, grad_fn=<AddBackward0>) tensor(29051.3965, grad_fn=<AddBackward0>)
tensor(7736.3047, grad_fn=<SumBackward0>) tensor(6462795.5000, grad_fn=<AddBackward0>) tensor(6463392.5000, grad_fn=<AddBackward0>)
tensor(5885.4551, grad_fn=<SumBackward0>) tensor(-666.3966, grad_fn=<AddBackward0>) tensor(-89.9656, grad_fn=<AddBackward0>)
tensor(5586.4653, grad_fn=<SumBackward0>) tensor(-441.5450, grad_fn=<AddBackward0>) tensor(129.4318, grad_fn=<AddBackward0>)
tensor(7448.5063, grad_fn=<SumBackward0>) tensor(-625.1521, grad_fn=<AddBackward0>) tensor(-37.4009, grad_fn=<AddB